In [1]:
import os
import time
import random
from itertools import count, combinations

import chess
import pandas as pd
from tqdm import tqdm 

from IPython.display import clear_output, HTML

In [2]:
bot_dir = 'bots'
bot_files = [bot_file for bot_file in os.listdir(bot_dir) if bot_file.endswith('.py')]
bot_files = sorted(bot_files)

bot_files

['JernoIsBot__J_Beuker.py',
 'Odysseus__m_lindenburger_1.py',
 'bot_number_piece.py',
 'bot_piece_value.py',
 'bot_random.py']

In [3]:
# import bots from the files

bot_classes = []

for bot_file in bot_files:
    bot_name = bot_file[:-3]
    bot_module = __import__('bots.' + bot_name, fromlist=[bot_name])
    bot_class = getattr(bot_module, "ChessBot")
    bot_class.name = bot_name
    
    bot_classes.append(bot_class)

print(bot_classes)

[<class 'bots.JernoIsBot__J_Beuker.ChessBot'>, <class 'bots.Odysseus__m_lindenburger_1.ChessBot'>, <class 'bots.bot_number_piece.ChessBot'>, <class 'bots.bot_piece_value.ChessBot'>, <class 'bots.bot_random.ChessBot'>]


In [4]:
class Judge():
    def __init__(self, player_1, player_2, time_limit=300): # 5 minutes per player
        self.player_1 = player_1            # white
        self.player_2 = player_2            # black
        self.time_limit = time_limit

    def run_game(self, initial_board_fen: str = None, slow_down: bool = False, verbose: bool = False):
        if initial_board_fen is None:
            initial_board_fen = chess.Board().fen()
        board = chess.Board(initial_board_fen)

        player_times = [0, 0]
        history = []
        notes = []
        winner = None

        for i in count(0, 1):
            history.append(board.fen())

            if board.is_checkmate():
                winner = "black" if i%2 == 0 else "white"
                notes.append(f"Checkmate! Winner is {winner}")
                break
            elif board.is_stalemate():
                notes.append("Stalemate! It's a draw.")
                break
            elif board.is_insufficient_material():
                notes.append("Insufficient material! It's a draw.")
                break
            elif board.is_seventyfive_moves():
                notes.append("Seventy-five moves without a capture or pawn move! It's a draw.")
                break
            elif board.is_fivefold_repetition():
                notes.append("Fivefold repetition! It's a draw.")
                break
            elif board.is_game_over():
                notes.append("Game over! It's a draw.")
                break

            board_fen = board.fen()

            start = time.time()
            if i % 2 == 0:
                move = self.player_1(board_fen)
            else:
                move = self.player_2(board_fen)
            end = time.time()

            player_times[i%2] += end - start

            if player_times[i%2] > self.time_limit:
                winner = "black" if i%2 == 0 else "white"
                notes.append(f"\nTime limit exceeded! Winner is {winner}")
                break

            if not board.is_legal(move):
                winner = "black" if i%2 == 0 else "white"
                hallucinator = "white" if i%2 == 0 else "black"
                notes.append(f"Illegal board move. The {hallucinator} is hallucinating.")
                break

            board.push(move)

            if verbose: 
                clear_output(wait=True)
                display(board)
                display(HTML(f"{self.player_1.name} (white) vs {self.player_2.name} (black)"))

            # slow down the bots so that we can see them
            if slow_down: 
                time.sleep(.001)

        results = {}
        results['winner'] = winner
        results['times'] = player_times
        results["notes"] = notes
        results["history"] = history

        return results

In [5]:
with open('opennings.txt', 'r') as file:
    opennings = file.read().split('\n')

In [6]:
# play all bots against each other

N_GAMES = 3 # per pair color (so N_GAMES * 2 games per pair of bots)

game_results = pd.DataFrame(columns=['white_bot', 'black_bot', 'winner', 'white_time', 'black_time', 'notes', 'history'])

combin = tqdm(combinations(bot_classes, 2))

for bot1, bot2 in combin:
    for white, black in [(bot1, bot2), (bot2, bot1)]:
        for i in range(N_GAMES):
            judge = Judge(white(), black())
            random_opening = random.choice(opennings)
            results = judge.run_game(random_opening, verbose=True)

            game_results.loc[len(game_results)] = [
                white.name, black.name, results['winner'], 
                results['times'][0], results['times'][1], 
                "; ".join(results['notes']),
                "\t".join(results['history'])
            ]

clear_output(wait=True)
display(game_results)

,white_bot,black_bot,winner,white_time,black_time,notes,history
0,JernoIsBot__J_Beuker,Odysseus__m_lindenburger_1,None,6.620246,0.498662,Fivefold repetition! It's a draw.,rnbqkbnr/pp1ppppp/2p5/8/4P3/8/PPPP1PPP/RNBQKBN...
1,JernoIsBot__J_Beuker,Odysseus__m_lindenburger_1,white,0.824387,0.081355,Checkmate! Winner is white,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...
2,JernoIsBot__J_Beuker,Odysseus__m_lindenburger_1,white,2.072977,0.111270,Checkmate! Winner is white,rnbqkbnr/ppppp1pp/8/5p2/3P4/8/PPP1PPPP/RNBQKBN...
3,Odysseus__m_lindenburger_1,JernoIsBot__J_Beuker,black,0.367033,8.356932,Checkmate! Winner is black,rnbqkbnr/ppp1pppp/3p4/8/3P4/8/PPP1PPPP/RNBQKBN...
4,Odysseus__m_lindenburger_1,JernoIsBot__J_Beuker,black,0.274016,4.659831,Checkmate! Winner is black,rnbqkbnr/ppp1pppp/8/3p4/8/5N2/PPPPPPPP/RNBQKB1...
5,Odysseus__m_lindenburger_1,JernoIsBot__J_Beuker,black,0.311699,4.405250,Checkmate! Winner is black,rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBN...
6,JernoIsBot__J_Beuker,bot_number_piece,white,4.330662,5.571939,Checkmate! Winner is white,rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBN...
7,JernoIsBot__J_Beuker,bot_number_piece,white,0.824565,1.545009,Checkmate! Winner is white,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...
8,JernoIsBot__J_Beuker,bot_number_piece,white,6.962077,7.929487,Checkmate! Winner is white,rnbqkbnr/pppp1ppp/8/4p3/2P5/8/PP1PPPPP/RNBQKBN...
9,bot_number_piece,JernoIsBot__J_Beuker,black,2.542158,2.040976,Checkmate! Winner is black,rnbqkb1r/pppppppp/5n2/8/2P5/8/PP1PPPPP/RNBQKBN...


In [10]:
leaderboard = pd.DataFrame(columns=['bot', 'wins', 'draws', 'losses', 'score'])

for bot in bot_classes:
    bot_name = bot.name

    bot_wins = sum((game_results["white_bot"] == bot_name) & (game_results["winner"] == "white")) + \
               sum((game_results["black_bot"] == bot_name) & (game_results["winner"] == "black"))
    
    bot_draws = sum((game_results["white_bot"] == bot_name) & (game_results["winner"].isna())) + \
                sum((game_results["black_bot"] == bot_name) & (game_results["winner"].isna()))
    
    bot_losses = sum((game_results["white_bot"] == bot_name) & (game_results["winner"] == "black")) + \
                    sum((game_results["black_bot"] == bot_name) & (game_results["winner"] == "white"))
    
    bot_score = bot_wins + 0.5 * bot_draws + 0 * bot_losses

    leaderboard.loc[len(leaderboard)] = [bot_name, bot_wins, bot_draws, bot_losses, bot_score]

leaderboard = leaderboard.sort_values(by='score', ascending=False)
leaderboard = leaderboard.reset_index(drop=True)

leaderboard

,bot,wins,draws,losses,score
0,JernoIsBot__J_Beuker,22,2,0,23.0
1,bot_random,2,16,6,10.0
2,bot_number_piece,1,17,6,9.5
3,Odysseus__m_lindenburger_1,1,16,7,9.0
4,bot_piece_value,0,17,7,8.5


In [11]:
today = time.strftime("%Y-%m-%d")
today_results = f"results/{today}_results.csv"
today_leaderboard = f"results/{today}_leaderboard.csv"

game_results.to_csv(today_results, index=False)
leaderboard.to_csv(today_leaderboard, index=False)

In [12]:
print(leaderboard.to_markdown())

|    | bot                        |   wins |   draws |   losses |   score |
|---:|:---------------------------|-------:|--------:|---------:|--------:|
|  0 | JernoIsBot__J_Beuker       |     22 |       2 |        0 |    23   |
|  1 | bot_random                 |      2 |      16 |        6 |    10   |
|  2 | bot_number_piece           |      1 |      17 |        6 |     9.5 |
|  3 | Odysseus__m_lindenburger_1 |      1 |      16 |        7 |     9   |
|  4 | bot_piece_value            |      0 |      17 |        7 |     8.5 |
